In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys 
sys.path.insert(0, 'E:/repos/my_projects/taxi_demand_predictor')

In [3]:
from dotenv import load_dotenv
import os
from src.paths import PATRENT_DIR

In [4]:
HOPSWORKS_PROJECT = 'taxi_demand_salar'

In [5]:
load_dotenv(PATRENT_DIR/'.env')
HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [6]:
from datetime import datetime
from src.data import load_raw_data , transform_to_ts_data
import pandas as pd


In [7]:

from_date = 2022
end_date = datetime.now().year

print(f'downloading data from {from_date} to {end_date}')
rides_data =pd.DataFrame()

for year in range(from_date,end_date+1):
    rides_one_year = load_raw_data(year=year)

    rides_data = pd.concat([rides_data,rides_one_year])




downloading data from 2022 to 2023
file taxi_2022_01.parquet already downloaded.
file taxi_2022_02.parquet already downloaded.
file taxi_2022_03.parquet already downloaded.
file taxi_2022_04.parquet already downloaded.
file taxi_2022_05.parquet already downloaded.
file taxi_2022_06.parquet already downloaded.
file taxi_2022_07.parquet already downloaded.
downloading file taxi_2022_08.parquet
downloading file taxi_2022_09.parquet
downloading file taxi_2022_10.parquet
downloading file taxi_2022_11.parquet
downloading file taxi_2022_12.parquet
downloading file taxi_2023_01.parquet
downloading file taxi_2023_02.parquet
downloading file taxi_2023_03.parquet
downloading file taxi_2023_04.parquet
downloading file taxi_2023_05.parquet
file taxi_2023_05.parquet not available
downloading file taxi_2023_06.parquet
file taxi_2023_06.parquet not available
downloading file taxi_2023_07.parquet
file taxi_2023_07.parquet not available
downloading file taxi_2023_08.parquet
file taxi_2023_08.parquet not

In [8]:
rides_data.shape


(52326988, 2)

In [9]:
rides_data.head()

,pu_datetime,pu_location
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [10]:
rides_data = transform_to_ts_data(raw_data=rides_data)

100%|██████████| 262/262 [00:09<00:00, 27.84it/s]


In [11]:
rides_data.head()


,pu_hour,rides,pu_location
0,2022-01-01 00:00:00,11,4
1,2022-01-01 01:00:00,15,4
2,2022-01-01 02:00:00,26,4
3,2022-01-01 03:00:00,8,4
4,2022-01-01 04:00:00,9,4


In [12]:
import hopsworks

In [13]:
project = hopsworks.login(
    project= HOPSWORKS_PROJECT,
    api_key_value= HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.2) by running 'pip install hopsworks==3.2.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/69786


In [14]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [15]:
FEATURE_GROUP_NAME = 'ts_data_feature_group'
FEATURE_GROUP_VERSION = 1

In [16]:
feature_group = feature_store.get_or_create_feature_group(
    name = FEATURE_GROUP_NAME,
    version= FEATURE_GROUP_VERSION,
    description= "Time Series Data with Hourly frequency",
    primary_key= ['pu_location','pu_hour'],
    event_time= 'pu_hour'
)

In [17]:
feature_group.insert(rides_data, write_options={'wait_for_job':False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/69786/fs/69705/fg/73089


Uploading Dataframe: 0.00% |          | Rows 0/3049680 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/69786/jobs/named/ts_data_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x28d76d6b850>, None)